<a href="https://colab.research.google.com/github/yuuuwoo/Video_tracking788/blob/main/Final_test_tfrecord(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/mydrive')

Drive already mounted at /content/mydrive; to attempt to forcibly remount, call drive.mount("/content/mydrive", force_remount=True).


In [ ]:
# read costumized file path
%cd mydrive/MyDrive/Colab_Notebooks/SEP788GP3/SiameseFC-tf
!ls

/content/mydrive/MyDrive/Colab_Notebooks/SEP788GP3/SiameseFC-tf
BlurCar1			 README.md
get_shuffled_list_from_vedio.py  requirements.txt
output				 run_tracker_evaluation.py
parameters			 run_tracker_training.py
prepare_training_dataset.py	 src


#Import

In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
%matplotlib inline
import glob
from random import shuffle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


#Data processing


For gif or mp4 files, usually slice each frame into image sequence\
For our dataset, it is already sliced into multiple frame images and one txt label. \
So that we can save a step to slice the video frame

In [ ]:
cur_dir = os.getcwd()
object_floder = os.path.join(cur_dir, 'BlurCar1')#choose object
print(object_floder)

/content/mydrive/MyDrive/Colab_Notebooks/SEP788GP3/SiameseFC-tf/BlurCar1


##Read data 

In [ ]:
def load_images_from_folder(image_folder):
  images = []
  images_filename = []
  lst = os.listdir(image_folder)
  lst.sort()

  for file in lst:
        img = cv2.imread(os.path.join(image_folder,file))#height, width, channel
        if img is not None:
            images.append(img)
            images_filename.append(file)
            h, w , c = img.shape
            
        
  return images, images_filename, h, w, c

In [ ]:
from pandas._libs import index
#blur car number#
label_folder = os.path.join(object_floder, 'groundtruth_rect.txt')
image_folder = os.path.join(object_floder,'img')

#Read from groundtruth.txt
head = ['x', 'y', 'box_width', 'box_height']
labeldata = pd.read_csv(label_folder, sep= '\s+', names = head)
labeldata['x_center']= np.nan
labeldata['y_center']= np.nan

#Read image 
images, images_filename, h , w, c = load_images_from_folder(image_folder)
imagedata = pd.DataFrame(list(zip(images_filename,images)), columns=['filename', 'imread'])

#caculate x_center and y_center, normalize with image size

x_center = []
y_center = []
for i in range(len(imagedata)):
  x = labeldata.iloc[i,0]
  y = labeldata.iloc[i,1]
  width = labeldata.iloc[i,2]
  height = labeldata.iloc[i,3]
  x_cen = (x + width)/2
  y_cen = (y + height)/2
  #write normalized 
  labeldata.iloc[i,4] = x_cen/w
  labeldata.iloc[i,5] = y_cen/h
  labeldata.iloc[i,2] = width/w
  labeldata.iloc[i,3] = height/h

#save to dataframe
df = labeldata.join(imagedata)
df = df[['x_center', 'y_center','box_width','box_height', 'filename','imread']]
df.head()


,x_center,y_center,box_width,box_height,filename,imread
0,0.278125,0.284375,0.165625,0.218750,0247.jpg,"[[[255, 235, 183], [255, 235, 183], [255, 239,..."
1,0.277344,0.287500,0.165625,0.220833,0248.jpg,"[[[255, 236, 181], [255, 237, 182], [255, 238,..."
2,0.271875,0.296875,0.168750,0.220833,0249.jpg,"[[[255, 232, 176], [255, 232, 176], [254, 234,..."
3,0.257812,0.314583,0.167187,0.231250,0250.jpg,"[[[255, 238, 181], [255, 238, 181], [255, 239,..."
4,0.256250,0.327083,0.181250,0.218750,0251.jpg,"[[[255, 235, 180], [255, 234, 179], [255, 235,..."


##Prepare for simese by use tf.record
label format:\
z for examplar image, x for search image\
z_pos_x is x_center\
z_pos_y is x_center\
z_target_w is box width\
z_target_h is box height\
\
x_pos_x is ...


In [ ]:
#generate shuffle examplar list and search list to txt


def prepare_shuffled_list(df, output_directory, output_filename):
  if not os.path.exists(output_directory) or os.path.isfile(output_directory):
    os.makedirs(output_directory)

  output_list = []
  row_id = df.index.tolist()
  for i in range(len(row_id) - 1):
    z_path = os.path.join(image_folder, df.iloc[i].filename)
    x_path = os.path.join(image_folder, df.iloc[i+1].filename)
    z_pos_x = df.iloc[i].x_center
    z_pos_y = df.iloc[i].y_center
    z_target_w = df.iloc[i].box_width
    z_target_h = df.iloc[i].box_height
    x_pos_x = df.iloc[i+1].x_center
    x_pos_y = df.iloc[i+1].y_center
    x_target_w = df.iloc[i+1].box_width
    x_target_h = df.iloc[i+1].box_height
    output_list.append(z_path + " " + x_path + " " 
                       + str(z_pos_x)+ " "+ str(z_pos_y)+ " "+  str(z_target_w)+ " "+ str(z_target_h)+ " "
                       +  str(x_pos_x)+ " "+ str(x_pos_y)+ " "+  str(x_target_w)+ " "+  str(x_target_h))

  shuffle(output_list)
  with open(os.path.join(output_directory,output_filename) + ".txt","w+") as f:
        for output_file in output_list:
            f.write(output_file + "\n") 
  print("Done. tfrecords file saved at " + output_directory)


  

In [ ]:
output_directory = os.path.join(object_floder, "shuffled_data_list")
output_filename = "shuffled_list"

prepare_shuffled_list(df, output_directory, output_filename)


Done. tfrecords file saved at /content/mydrive/MyDrive/Colab_Notebooks/SEP788GP3/SiameseFC-tf/BlurCar1/shuffled_data_list


In [ ]:
#Write infomations to tf.record
from src.parse_arguments import parse_arguments
def transform2tfrecord(data_file, tfrecord_name, output_directory, resize_width, resize_height):
    """
        Input:
            data_folder: relative path of folder who contains all the vedio folders for training.
            tfrecord_name: nameof the tfrecord file
            output_directory: relative dir which will contain the tfrecord file
    """
    if not os.path.exists(output_directory) or os.path.isfile(output_directory):
        os.makedirs(output_directory)
        
    #init tfrecord file
    filename = os.path.join(output_directory, tfrecord_name + '.tfrecords')
    writer = tf.io.TFRecordWriter(filename)
        
    cur_dir = os.getcwd()
    data_folder = os.path.join(cur_dir, data_file)
    with open(data_folder, "r") as f:
        print("Generating tfrecord from " + data_folder)
        data_list = f.readlines()
        for data in data_list:
            z, x, z_pos_x, z_pos_y, z_target_w, z_target_h, x_pos_x, x_pos_y, x_target_w, x_target_h = data.strip("\n ").split(" ")
            z_pos_x = float(z_pos_x)
            z_pos_y = float(z_pos_y)
            z_target_w = float(z_target_w)
            z_target_h = float(z_target_h)
            x_pos_x = float(x_pos_x)
            x_pos_y = float(x_pos_y)
            x_target_w = float(x_target_w)
            x_target_h = float(x_target_h)
            z_img = cv2.imread(z)
            z_img = cv2.resize(z_img, (resize_width,resize_height))
            x_img = cv2.imread(x)
            x_img = cv2.resize(x_img, (resize_width,resize_height))
            z_raw = z_img.tostring()
            x_raw = x_img.tostring()
            

            example = tf.train.Example(features=tf.train.Features(feature={
                'z_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[z_raw])),
                'x_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[x_raw])),
                'z_pos_x': tf.train.Feature(float_list=tf.train.FloatList(value=[z_pos_x])),
                'z_pos_y': tf.train.Feature(float_list=tf.train.FloatList(value=[z_pos_y])),
                'z_target_w': tf.train.Feature(float_list=tf.train.FloatList(value=[z_target_w])),
                'z_target_h': tf.train.Feature(float_list=tf.train.FloatList(value=[z_target_h])),
                'x_pos_x': tf.train.Feature(float_list=tf.train.FloatList(value=[x_pos_x])),
                'x_pos_y': tf.train.Feature(float_list=tf.train.FloatList(value=[x_pos_y])),
                'x_target_w': tf.train.Feature(float_list=tf.train.FloatList(value=[x_target_w])),
                'x_target_h': tf.train.Feature(float_list=tf.train.FloatList(value=[x_target_h]))
                
            }))
            writer.write(example.SerializeToString())
        
        
 
        
    writer.close()
    print("Writer closed.")
    print(tfrecord_name + '.tfrecords'+" is written to "+output_directory)



In [ ]:
hp, evaluation, run, env, design = parse_arguments()#Reads the parameters .json file 

transform2tfrecord("./BlurCar1/shuffled_data_list/shuffled_list.txt", "training_dataset", "./BlurCar1/tfrecords", 
                  resize_width = design.resize_width, resize_height = design.resize_height)

Generating tfrecord from /content/mydrive/MyDrive/Colab_Notebooks/SEP788GP3/SiameseFC-tf/./BlurCar1/shuffled_data_list/shuffled_list.txt


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.


Writer closed.
training_dataset.tfrecords is written to ./BlurCar1/tfrecords


#Trainer